In [1]:
import os
from smolagents import AzureOpenAIServerModel

model = AzureOpenAIServerModel(
    model_id=os.environ.get("AZURE_OPENAI_MODEL", ""),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY", ""),
    api_version=os.environ.get("OPENAI_API_VERSION", "")
)

In [2]:
from pathlib import Path
from medminer.data import Document

docs = []
for file in (Path.cwd().parent / "data" / "icu_course").glob("*.txt"):
    with open(file, "r") as f:
        docs.append(Document(file.stem, f.read()))

In [ ]:
from medminer.utils.models import DefaultModel
from medminer.task.procedure import ProcedureTask

task = ProcedureTask(DefaultModel().model)

for doc in docs:
    task.run(doc.content)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Task name: procedure                                                                                            │
│ Prompt:                                                                                                         │
│     Given a medical course of a patient, extract all given procedures and save all information as csv. The      │
│ procedures can be in any language. The medical course is usually in the format of bullet points. Every          │
│ procedure should have a single row, if there are multiple procedures that can be extracted from a single piece  │
│ of text, split them up. These are the steps you should follow to complete the task:                             │
│                                                                                                                 │
│     1. Extract a part of the text that contains a procedure. The procedure can be in any language. This is the  │
│ `procedure_reference` column.                                                                                   │
│     2. If you suspect that the procedure_reference is an acronym, try to expand it from the context of the note │
│ you are working on. Do this in the original language. If you are not sure, leave it as is. This is the          │
│ `procedure_expanded` column. If it is not an acronym, you can skip this step.                                   │
│     3. Translate the procedures to English if necessary and infer the `procedure_translated` column. Use the    │
│ procedure_expanded column if it is available. If not, use the `procedure_reference` column. Strictly avoid      │
│ using acronyms in this column. This is the `procedure_translated` column.                                       │
│     4. Extract the relevant procedure as a string and remove everything that is not relevant. This is the       │
│ `procedure` column.                                                                                             │
│     5. Extract the date of the procedure. If not applicable, write an empty string. This is the `date` column.  │
│     6. Use the `search_snomed_procedures` tool to find SNOMED CT concepts for the extracted procedures. Add the │
│ SNOMED CT ID and fully specified name (FSN) to the output. You will get back a list of dictionaries with the    │
│ following keys: ids and fsn. Usually the first candidate is the best choice, but you can decide otherwise if    │
│ you have reasonable grounds for another decision if you compare the extracted information with the returned     │
│ descriptions from the snomed server. If there are no codes, write an empty string. This is the `snomed_id` and  │
│ `snomed_fsn` columns.                                                                                           │
│                                                                                                                 │
│     Example 1:                                                                                                  │
│     Input: "Colonoscopy performed on 2023-04-15"                                                                │
│     Output: [                                                                                                   │
│         {"patient_id": 1, "procedure_reference": "Colonoscopy performed on 2023-04-15",                         │
│         "procedure_expanded": "", "procedure_translated": "Colonoscopy", "procedure": "Colonoscopy", "date":    │
│ "2023-04-15", "snomed_id": "73761001", "snomed_fsn": "Colonoscopy (procedure)"},                                │
│         \]                                                                                                      │
│                                                       

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━